Data Exploration

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import *
from keras.optimizers import Adam
from keras.models import Sequential
from keras import Model

Loading data

In [2]:
#Reading all the images into a numpy array
images = []
for file in os.listdir():
  if file.endswith('.jpg'):
    #if the file is an image
    image_values = cv2.imread(file)

    #resize the image into 360 x 480 pixels
    image_values = cv2.resize(image_values,(360,480))

    images.append(image_values)

#storing everything in a numpy array
images_array = np.array(images)

Finding number of observations

In [3]:
number_of_observations = len(images_array)
number_of_observations

31

As we are working on a image dataset we do not have any columns 

Analzing how images are stored in python variables

In [4]:
images_array

array([[[[ 39,  46,  55],
         [ 52,  59,  68],
         [ 80,  87,  96],
         ...,
         [ 11,   6,   7],
         [ 11,   6,   7],
         [ 11,   6,   7]],

        [[ 35,  42,  51],
         [ 48,  55,  64],
         [ 74,  81,  90],
         ...,
         [ 10,   6,   6],
         [ 11,   6,   6],
         [ 11,   5,   6]],

        [[ 24,  31,  40],
         [ 36,  43,  52],
         [ 59,  66,  75],
         ...,
         [  9,   5,   6],
         [  9,   5,   6],
         [ 10,   5,   6]],

        ...,

        [[ 57,  61,  66],
         [ 45,  49,  54],
         [ 21,  23,  28],
         ...,
         [ 26,  32,  30],
         [ 24,  29,  28],
         [ 23,  28,  27]],

        [[104, 108, 113],
         [ 83,  87,  92],
         [ 41,  44,  49],
         ...,
         [ 27,  31,  31],
         [ 24,  29,  28],
         [ 23,  28,  27]],

        [[123, 127, 132],
         [ 98, 103, 107],
         [ 49,  53,  57],
         ...,
         [ 27,  32,  31],
        

In [5]:
shape = images_array[0].shape
shape

(480, 360, 3)

In [6]:
images_array[0]

array([[[ 39,  46,  55],
        [ 52,  59,  68],
        [ 80,  87,  96],
        ...,
        [ 11,   6,   7],
        [ 11,   6,   7],
        [ 11,   6,   7]],

       [[ 35,  42,  51],
        [ 48,  55,  64],
        [ 74,  81,  90],
        ...,
        [ 10,   6,   6],
        [ 11,   6,   6],
        [ 11,   5,   6]],

       [[ 24,  31,  40],
        [ 36,  43,  52],
        [ 59,  66,  75],
        ...,
        [  9,   5,   6],
        [  9,   5,   6],
        [ 10,   5,   6]],

       ...,

       [[ 57,  61,  66],
        [ 45,  49,  54],
        [ 21,  23,  28],
        ...,
        [ 26,  32,  30],
        [ 24,  29,  28],
        [ 23,  28,  27]],

       [[104, 108, 113],
        [ 83,  87,  92],
        [ 41,  44,  49],
        ...,
        [ 27,  31,  31],
        [ 24,  29,  28],
        [ 23,  28,  27]],

       [[123, 127, 132],
        [ 98, 103, 107],
        [ 49,  53,  57],
        ...,
        [ 27,  32,  31],
        [ 24,  29,  28],
        [ 23,  28,  27]]

Here we can observe that the images are stored pxiel wise as a 2d matrix where each element in the matrix is an array with 3 values in it. Those three values are RGB values(red green blue) of the pixel 

There are no missing values

All the pixel details in the image dataset are used as features for building our dep learning model

Target column: Our model wil be creating new face images by taking inspiration from the images we feed it. We also train a discriminator which identifies whether an image is a real one or generated one

**Steps to be performed in the project**

* I will be loading the dataset into numpy arrays. 
* I will preprocess the images like fitting its resolution. 
* I will be creating a GAN model and training it with the dataset I loaded. 
* I will be training a discriminator that can filter whether an image is real or generated
* Hyper parameter tuning will be performed
* I will be creating new images using the generator.
* For evaluation purpose, I will be taking a model available on internet to predict if generated image is a human face or not




In [7]:
# building the discriminator
def build_discriminator(img_shape):
  model = Sequential([
      # First Layer
      Conv2D(32, kernel_size=5, strides=2, input_shape=img_shape, padding="same"),                          
      LeakyReLU(alpha=0.2),                           
      Dropout(0.25),
      # Second Layer
      Conv2D(64, kernel_size=5, strides=2, padding="same"),                          
      BatchNormalization(momentum=0.8),                           
      LeakyReLU(alpha=0.2),                           
      Dropout(0.25),
      # Third layer
      Conv2D(128, kernel_size=5, strides=2, padding="same"),                         
      BatchNormalization(momentum=0.8),                           
      LeakyReLU(alpha=0.2),                           
      Dropout(0.25),
      # Flattening the output to get a valid value
      Flatten(),                            
      Dense(1),                           
      Activation("sigmoid")
  ])
                                                    
  model.summary()                           
  img = Input(shape=img_shape)    
  # predicted outputs
  d_pred = model(img)                           
  return Model(inputs=img, outputs=d_pred)

In [8]:
# Building the Generator
def build_generator(z_dimension, channels):                           
  model = Sequential([
      Dense(2 * 120 * 90, input_dim=z_dimension),                           
      LeakyReLU(alpha=0.2),                            
      Reshape((120, 90, 2)),
      UpSampling2D(),                          
      # Second Layer
      Conv2D(128, kernel_size=5, padding="same"),                           
      BatchNormalization(momentum=0.8),                           
      LeakyReLU(alpha=0.2),
      UpSampling2D(),                          
      # Third Layer
      Conv2D(64, kernel_size=5, padding="same"),                           
      BatchNormalization(momentum=0.8),                           
      LeakyReLU(alpha=0.2),
      Conv2D(channels, kernel_size=5, padding="same"),                           
      Activation("tanh"),
  ])
  model.summary()                           
  noise = Input(shape=(z_dimension,))                           
  img = model(noise)                           
  return Model(inputs=noise, outputs=img)

In [9]:
#load real pictures:                       
x_train = images_array                                       
# model parameters                       
img_rows = shape[0]                       
img_cols = shape[1]                       
channels = 3                       
img_shape = (img_rows, img_cols, channels)                       
z_dimension = 32        
# Using adam optimiser here because the computation is fast and results are generally better
optimizer = Adam(0.0005, 0.5)

In [10]:
# Compiling the discriminator and generator
discriminator = build_discriminator(img_shape)      
discriminator.compile(loss='binary_crossentropy',                                             
                      optimizer=optimizer, metrics=['accuracy'])                                               
generator = build_generator(z_dimension,channels)                                               
z = Input(shape=(z_dimension,))                       
img = generator(z)                       
discriminator.trainable = False                       
d_pred = discriminator(img)                       
combined = Model(z, d_pred)                       
combined.compile(loss='binary_crossentropy',optimizer=optimizer,                                        
                 metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 180, 32)      2432      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 240, 180, 32)      0         
                                                                 
 dropout (Dropout)           (None, 240, 180, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 90, 64)       51264     
                                                                 
 batch_normalization (BatchN  (None, 120, 90, 64)      256       
 ormalization)                                                   
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 120, 90, 64)       0         
                                                        

In [ ]:
# Total number of epochs to train for
epochs = 2

# Number of batches per epoch
batch_size = 64
sample_interval=1000 

# Real labels
real = np.ones((batch_size, 1))
# generated labels
fake = np.zeros((batch_size, 1))

# Running it for 2 epochs
for epoch in range(epochs):
    # Select a batch of random real images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    # Fetch the images from the training set
    imgs = x_train[idx]
    noise = np.random.normal(0, 1, (batch_size, z_dimension))
    # Generate fake images
    gen_imgs = generator.predict(noise)
    # Training the generator to find the loss
    d_loss_real = discriminator.train_on_batch(imgs, real)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    noise = np.random.normal(0, 1, (batch_size, z_dimension))
    g_loss = combined.train_on_batch(noise, real)
    # Printing the loss every epoch
    if (epoch % sample_interval) == 0:
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % 
            (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))
        generator.save("generator_64_64_z64_%d_epoch.h5" % epoch)

2/2 [==============================] - 87s 43s/step
